## Data Exploration - Outlier Detection

#### Objective: Ability to read clean and detect outliers in data

In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv('/Users/admin/Documents/VS Code Learning/DataExplorationProjects/DataOutlierDetectionsCSV/raw_data.csv')

In [3]:
df.sample(n=10, random_state=20)

,member_unique_id,gender,dob,eligible_year,eligible_month,affiliation_type,pbp_group,plan_name,npi,line_of_business,...,pcp_ffs,plan_premium,prof,reinsurance,risk_score_partd,rx,rx_rebates,rx_with_rebates,rx_without_rebates,spec_cap
7458,7459,F,08/12/1933,2020,202006,VIPcare,NON-SNP,NaN,63,HMO,...,$0.00,"$20,200.00",$0.00,$8.06,$0.00,$12.06,$0.00,$13.21,$13.14,$786.16
8047,8048,F,06/05/1961,2020,202006,Affiliate,NON-SNP,MEDICARE - CAREFREE,69,HMO,...,$0.00,"$15,326.47",$0.00,$9.39,$0.00,"$69,247.49",$0.00,"$68,915.21","$71,028.34",$909.74
8052,8053,M,25/12/1949,2020,202006,Affiliate,NON-SNP,MEDICARE - CAREFREE,69,HMO,...,$0.00,"$16,007.97",$0.00,$8.21,$0.00,$0.00,$0.00,$0.00,$0.00,$897.84
2522,2523,F,10/02/1953,2020,202006,Affiliate,NON-SNP,MEDICARE - CAREFREE,20,HMO,...,$0.00,"$4,177.98",$730.58,$9.20,$0.00,$289.80,$0.00,$291.99,$293.65,$866.42
4113,4114,M,09/11/1948,2020,202006,Affiliate,NON-SNP,NaN,31,HMO,...,$0.00,"$10,032.17",$0.00,$8.51,$0.00,"$15,681.50",$0.00,"$13,977.64","$14,058.75",$980.47
7347,7348,F,09/04/1952,2020,202006,Affiliate,NON-SNP,MEDICARE - CAREFREE,62,HMO,...,$0.00,"$3,067.57",$0.00,$9.63,$0.00,$0.00,$0.00,$0.00,$0.00,$629.54
569,570,M,21/12/1946,2020,202006,Affiliate,NON-SNP,NaN,5,HMO,...,$0.00,"$8,089.97",$0.00,$9.38,$0.00,$454.15,$0.00,$436.94,$453.67,$770.56
6491,6492,M,20/04/1941,2020,202006,VIPcare,NON-SNP,NaN,57,HMO,...,$0.00,"$7,075.97",$0.00,$9.41,$0.00,$95.87,$0.00,$95.12,$95.02,$675.93
8630,8631,F,26/09/1946,2020,202006,VIPcare,NON-SNP,MEDICARE - CAREFREE,75,HMO,...,$0.00,"$2,584.05",$0.00,$9.24,$0.00,$458.20,$0.00,$474.33,$485.74,$659.55
9942,9943,F,09/04/1949,2020,202006,VIPcare,D-SNP,MEDICARE - CARENEEDS,87,HMO,...,$0.00,"$7,819.41",$0.00,$9.33,$0.00,$233.86,$0.00,$210.18,$239.30,$692.63


In [4]:
df.shape

(10530, 28)